In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math


subject_code = 'L'

df_RRI = pd.DataFrame()
df_RRI['reading1'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_reading1.csv")['RRI']
#df_RRI['reading2'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_reading2.csv")['RRI']
df_RRI['u-kt1'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_u-kt1.csv")['RRI']
#df_RRI['u-kt2'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_u-kt2.csv")['RRI']
df_RRI['task1'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_task1.csv")['RRI']
df_RRI['task2'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_task2.csv")['RRI']
df_RRI['task3'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_task3.csv")['RRI']
df_RRI['task4'] = pd.read_csv("../data/" + subject_code + "/RRI/RRI_task4.csv")['RRI']


df_base_params = pd.DataFrame()


df_RRI

,reading1,u-kt1,task1,task2,task3,task4
0,975.000000,824.000000,1692.000000,963.000000,689.000000,783.000000
1,932.540927,872.652655,830.011921,868.013000,687.703805,813.537028
2,915.714263,880.142019,847.187651,766.338297,725.689735,828.133124
3,880.301960,861.069380,854.311292,802.058614,752.115418,827.697441
4,843.311416,918.834766,876.454194,851.332097,809.071984,815.714060
...,...,...,...,...,...,...
279,895.483911,736.995650,NaN,NaN,NaN,NaN
280,894.962094,762.959665,NaN,NaN,NaN,NaN
281,940.262159,777.058694,NaN,NaN,NaN,NaN
282,949.660025,744.272110,NaN,NaN,NaN,NaN


# 基準値の算出 #

## ○LP面積

In [2]:
from hrv import LP_cal

calc_window = 60
slide_window = 1
LPS_list = []
CSI_list = []

for task in df_RRI.columns:
    RRI = df_RRI[task]
    LPS = []
    CSI = []
    i = calc_window+30 #初めの30秒をカット
    while i < len(RRI):
        if math.isnan(RRI.iloc[i]):
            break
        lps, csi, cvi = LP_cal(RRI.iloc[i-calc_window:i])
        LPS.append(lps)
        CSI.append(csi)
        i += slide_window
    LPS_list.append(sum(LPS)/len(LPS))
    CSI_list.append(sum(CSI)/len(CSI))
    
df_base_params['LPS'] = LPS_list
df_base_params['CSI'] = CSI_list
df_base_params.head()

,LPS,CSI
0,4.111056e+03,3.144586
1,4.037178e+03,3.611065
2,1.866040e+06,2.921270
3,1.781715e+04,2.451753
4,4.942706e+03,2.741982


## ○HR

In [3]:
from hrv import HR_cal
HR_list = []
for task in df_RRI.columns:
    i = 30
    RRI = df_RRI[task]
    HR = []
    while i<len(RRI):
        if math.isnan(RRI.iloc[i]):
            break
        HR.append(60000/RRI.iloc[i])
        i += 1  
    HR_list.append(sum(HR)/len(HR))
print(HR_list)
df_base_params['HR'] = HR_list
df_base_params

[65.59310864293849, 78.05554055704818, 43.77195712979135, 71.87656995927647, 75.28819992708522, 71.46985404776909]


,LPS,CSI,HR
0,4.111056e+03,3.144586,65.593109
1,4.037178e+03,3.611065,78.055541
2,1.866040e+06,2.921270,43.771957
3,1.781715e+04,2.451753,71.876570
4,4.942706e+03,2.741982,75.288200
5,4.165532e+03,2.977302,71.469854


## ○SDNN

In [4]:

calc_window = 60
slide_window = 1
SDNN_list = []

for task in df_RRI.columns:
    RRI = df_RRI[task]
    SDNN = []
    i = calc_window + 30
    while i < len(RRI):
        if math.isnan(RRI.iloc[i]):
            break
        SDNN.append(np.std(RRI.iloc[i-calc_window:i]))
        i += slide_window
    SDNN_list.append(sum(SDNN)/len(SDNN))
df_base_params['SDNN'] = SDNN_list
df_base_params

,LPS,CSI,HR,SDNN
0,4.111056e+03,3.144586,65.593109,50.724825
1,4.037178e+03,3.611065,78.055541,46.389653
2,1.866040e+06,2.921270,43.771957,1024.339097
3,1.781715e+04,2.451753,71.876570,81.139606
4,4.942706e+03,2.741982,75.288200,46.814263
5,4.165532e+03,2.977302,71.469854,49.517870


## ○pNN50

In [5]:

calc_window = 60
slide_window = 1
pNN50_list = []
RMSSD_list = []

for task in df_RRI.columns:
    RRI = df_RRI[task]
    pNN50 = []
    RMSSD = []
    diff = []
    i = calc_window + 30
    while i < len(RRI):
        if math.isnan(RRI.iloc[i]):
            break
        pnncnt = 0
        rmssd = 0
        for j in range(i-calc_window, i-1):
            diff.append(RRI[j+1]-RRI[j])
            if abs(diff[-1])>=50:
                pnncnt += 1
            rmssd += diff[-1]**2          
        pNN50.append(pnncnt/calc_window)
        RMSSD.append(math.sqrt(rmssd/(calc_window-1)))
        i += slide_window
    pNN50_list.append(sum(pNN50)/len(pNN50))
    RMSSD_list.append(sum(RMSSD)/len(RMSSD))
df_base_params['pNN50'] = pNN50_list
df_base_params['RMSSD'] = RMSSD_list
df_base_params

,LPS,CSI,HR,SDNN,pNN50,RMSSD
0,4.111056e+03,3.144586,65.593109,50.724825,0.305069,48.655184
1,4.037178e+03,3.611065,78.055541,46.389653,0.097938,39.242524
2,1.866040e+06,2.921270,43.771957,1024.339097,0.649311,886.262329
3,1.781715e+04,2.451753,71.876570,81.139606,0.248030,88.008307
4,4.942706e+03,2.741982,75.288200,46.814263,0.135256,44.760213
5,4.165532e+03,2.977302,71.469854,49.517870,0.208235,44.442556


#○CVRR#

In [6]:

CVRR_list = []
calc_window = 60
i = calc_window
for task in df_RRI.columns:
    RRI = df_RRI[task]
    CVRR = []
    i = calc_window + 30
    while i < len(RRI):
        if math.isnan(RRI[i]):
            break
        m_rri = np.mean(RRI)
        cvrr = 0
        for j in range(i-calc_window, i-1):
            cvrr += (RRI[j]-m_rri)**2
        cvrr = np.sqrt(cvrr/calc_window)/m_rri
        CVRR.append(cvrr)
        i+=1
    CVRR_list.append(sum(CVRR)/len(CVRR))
df_base_params['CVRR'] = CVRR_list
df_base_params

,LPS,CSI,HR,SDNN,pNN50,RMSSD,CVRR
0,4.111056e+03,3.144586,65.593109,50.724825,0.305069,48.655184,0.055482
1,4.037178e+03,3.611065,78.055541,46.389653,0.097938,39.242524,0.075340
2,1.866040e+06,2.921270,43.771957,1024.339097,0.649311,886.262329,0.891793
3,1.781715e+04,2.451753,71.876570,81.139606,0.248030,88.008307,0.108800
4,4.942706e+03,2.741982,75.288200,46.814263,0.135256,44.760213,0.065396
5,4.165532e+03,2.977302,71.469854,49.517870,0.208235,44.442556,0.069468


## ○LF,HF,LF/HF

In [7]:
from hrv import psd_cal
from hrv import calc_LF_HF

HF_list = []
LF_list = []
LF_HF_list = []
for task in df_RRI.columns:
    RRI = []
    i = 0
    for rri in df_RRI[task]:
        if math.isnan(rri):
            break
        RRI.append(rri)
    P, freq = psd_cal(RRI)
    vlf, lf, hf = calc_LF_HF(P, freq)
    HF_list.append(hf)
    LF_list.append(lf)
    LF_HF_list.append(lf/hf)
df_base_params['HF'] = HF_list
df_base_params['LF'] = LF_list
df_base_params['LF/HF'] = LF_HF_list
df_base_params

C:\Users\admini\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:691: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPRECATION_WARN, FutureWarning)


,LPS,CSI,HR,SDNN,pNN50,RMSSD,CVRR,HF,LF,LF/HF
0,4.111056e+03,3.144586,65.593109,50.724825,0.305069,48.655184,0.055482,380.109097,720.419226,1.895296
1,4.037178e+03,3.611065,78.055541,46.389653,0.097938,39.242524,0.075340,248.811328,306.568802,1.232134
2,1.866040e+06,2.921270,43.771957,1024.339097,0.649311,886.262329,0.891793,139927.311810,118694.710339,0.848260
3,1.781715e+04,2.451753,71.876570,81.139606,0.248030,88.008307,0.108800,1188.750306,1036.401554,0.871841
4,4.942706e+03,2.741982,75.288200,46.814263,0.135256,44.760213,0.065396,591.653629,473.534633,0.800358
5,4.165532e+03,2.977302,71.469854,49.517870,0.208235,44.442556,0.069468,245.100806,338.832420,1.382421


In [8]:
df_base_params.index = ['reading1',  'u-kt1','task1', 'task2', 'task3', 'task4']
df_base_params

,LPS,CSI,HR,SDNN,pNN50,RMSSD,CVRR,HF,LF,LF/HF
reading1,4.111056e+03,3.144586,65.593109,50.724825,0.305069,48.655184,0.055482,380.109097,720.419226,1.895296
u-kt1,4.037178e+03,3.611065,78.055541,46.389653,0.097938,39.242524,0.075340,248.811328,306.568802,1.232134
task1,1.866040e+06,2.921270,43.771957,1024.339097,0.649311,886.262329,0.891793,139927.311810,118694.710339,0.848260
task2,1.781715e+04,2.451753,71.876570,81.139606,0.248030,88.008307,0.108800,1188.750306,1036.401554,0.871841
task3,4.942706e+03,2.741982,75.288200,46.814263,0.135256,44.760213,0.065396,591.653629,473.534633,0.800358
task4,4.165532e+03,2.977302,71.469854,49.517870,0.208235,44.442556,0.069468,245.100806,338.832420,1.382421


In [9]:
reading = 'reading1'
ukt = 'u-kt1'

df_base = df_base_params.loc[[reading, ukt]]
df_base.index = ['reading', 'u-kt']
df_base.to_csv('../data/' + subject_code + '/base_params.csv')

# 正規化

In [10]:
reading_params = df_base_params.loc[reading]
ukt_params = df_base_params.loc[ukt]

task_name = ['task1', 'task2', 'task3', 'task4']
eda = []
scr = []
df_params = pd.DataFrame()
for param in df_base_params.columns:
    param_list = []
    for task in task_name:
        if param=='CSI' or param=='LF/HF':
            norm_param = (ukt_params[param]-df_base_params.loc[task][param])/(ukt_params[param]-reading_params[param])
        else:
            norm_param = (reading_params[param]-df_base_params.loc[task][param])/(reading_params[param]-ukt_params[param])
        param_list.append(norm_param)
    df_params[param] = param_list
for task in task_name:    
    df_eda = pd.read_csv("../data/" + subject_code + "/EDA/EDA_" + task + ".csv", index_col = 0)
    eda.append(np.mean(df_eda['filtered_eda']))
    df_scr = pd.read_csv("../data/" + subject_code + "/SCR/SCR_" + task + ".csv")
    scr.append(len(df_scr))
df_params['SCL'] = eda
df_params['SCR_peaknum'] = scr
df_params
df_params.index = ['task1', 'task2', 'task3', 'task4']
df_params

,LPS,CSI,HR,SDNN,pNN50,RMSSD,CVRR,HF,LF,LF/HF,SCL,SCR_peaknum
task1,-25202.639508,1.478726,-1.750955,-224.584932,-1.661959,-88.987300,42.115467,-1062.829962,-285.065049,-0.578854,0.391048,0
task2,-185.522582,2.485239,0.504192,-7.015819,0.275374,-4.180872,2.685005,-6.158834,-0.763518,-0.543294,0.703576,5
task3,-11.257027,1.863068,0.777945,0.902055,0.819832,0.413801,0.499250,-1.611181,0.596555,-0.651086,0.743882,8
task4,-0.737368,1.358609,0.471557,0.278410,0.467499,0.447549,0.704318,1.028260,0.922040,0.226622,0.336779,1


In [11]:
df_params.to_csv('../data/' + subject_code + '/normalized_params.csv', index=None)